In [1]:
from pynq import Overlay, MMIO
import numpy as np
from PIL import Image

# Load the bitstream onto the FPGA
overlay = Overlay("./design_1.bit")

# Base address and size
base_address = 0x00A0000000
image_size = 640 * 480  # Total number of pixels
total_bytes = image_size * 2  # Each RGB444 value is 2 bytes

# Create MMIO object to read from the specified address
mmio = MMIO(base_address, total_bytes)

# Read all BRAM content and save to file
with open('camera_444.bin', 'wb') as f:
    for i in range(0, total_bytes, 4):
        data = mmio.read(i)
        f.write(data.to_bytes(4, byteorder='little'))

# Read the data from the FPGA memory in chunks of 4 bytes
data = bytearray()
for i in range(0, total_bytes, 4):
    value = mmio.read(i, 4)
    data.extend(value.to_bytes(4, 'little'))

# Function to convert RGB444 to RGB888
def rgb444_to_rgb888(data):
    rgb888 = np.zeros((480, 640, 3), dtype=np.uint8)
    for i in range(480):
        for j in range(640):
            index = (i * 640 + j) * 2
            if index + 1 < len(data):
                rgb444 = (data[index + 1] << 8) | data[index]
                r = ((rgb444 >> 8) & 0xF) * 17
                g = ((rgb444 >> 4) & 0xF) * 17
                b = (rgb444 & 0xF) * 17
                rgb888[i, j] = [r, g, b]
    return rgb888

# Convert the data
rgb888_data = rgb444_to_rgb888(data)

# Create an image from the RGB data
image = Image.fromarray(rgb888_data, 'RGB')

# Save the image as a JPEG file
image.save("output_image.jpg")

In [2]:
from pynq import Overlay, MMIO
import numpy as np
from PIL import Image

# Load the bitstream onto the FPGA
overlay = Overlay("./design_1.bit")

# Base address and size
base_address = 0x00A0000000
image_width = 640
image_height = 480
num_pixels = image_width * image_height
total_bytes = num_pixels * 2  # Each RGB444 value is 2 bytes

# Create MMIO object to read from the specified address
mmio = MMIO(base_address, total_bytes)

# Read the data from the FPGA memory in chunks of 4 bytes
data = np.zeros(num_pixels, dtype=np.uint16)
for i in range(0, total_bytes, 4):
    value = mmio.read(i, 4)
    # Extract only the least significant 12 bits
    data[i // 2] = value & 0xFFF  # 12 bits

# Function to convert RGB444 to RGB888
def rgb444_to_rgb888(data, width, height):
    rgb888 = np.zeros((height, width, 3), dtype=np.uint8)
    for i in range(height):
        for j in range(width):
            index = i * width + j
            rgb444 = data[index]
            r = ((rgb444 >> 8) & 0xF) * 17
            g = ((rgb444 >> 4) & 0xF) * 17
            b = (rgb444 & 0xF) * 17
            rgb888[i, j] = [r, g, b]
    return rgb888

# Convert the data
rgb888_data = rgb444_to_rgb888(data, image_width, image_height)

# Create an image from the RGB data
image = Image.fromarray(rgb888_data, 'RGB')

# Save the image as a JPEG file
image.save("output_image.jpg")
